# Doing an exploration on recent disaster tweets for a disaster category

In [1]:
import pandas as pd
import pickle

import spacy
from spacy import displacy
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import networkx as nx 
import nx_altair as nxa
from collections import Counter
import warnings

from tools import extract_assign_location, disaster_title, topNlocations, top_locations, \
                    sampleNtweets, top_locations_tweets, show_top_locations, get_top_locations_flatten, \
                    get_location_graph_data, build_network, get_layout, plot_disaster_mentions, is_intersect, \
                    show_tweets_locations, top_location_pairs

warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', 280)

## Define any upstream dependencies

In [2]:
upstream = [
    'vectorizer',
    'category_classification_models',
    'recent_tweets_wildfire',
    'recent_tweets_earthquake',
    'recent_tweets_flood',
    'recent_tweets_hurricane'
]
random_seed = None
disaster_kind = None

In [3]:
# Parameters
random_seed = 42
disaster_kind = "recent_tweets_flood"
upstream = {
    "vectorizer": {
        "nb": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/vectorizer.ipynb",
        "vectorizer": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/vectorizer.pkl",
        "vocab": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/vocab.pkl",
    },
    "category_classification_models": {
        "nb": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/category_classification_models.ipynb",
        "model_lr": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/model_lr.pkl",
        "model_rf": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/model_rf.pkl",
        "model_nb": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/model_nb.csv",
        "model_votingc": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/model_votingc.csv",
    },
    "recent_tweets_earthquake": {
        "nb": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/recent_tweets_earthquake.ipynb",
        "file": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/recent_tweets_earthquake.csv",
    },
    "recent_tweets_wildfire": {
        "nb": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/recent_tweets_wildfire.ipynb",
        "file": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/recent_tweets_wildfire.csv",
    },
    "recent_tweets_hurricane": {
        "nb": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/recent_tweets_hurricane.ipynb",
        "file": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/recent_tweets_hurricane.csv",
    },
    "recent_tweets_flood": {
        "nb": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/recent_tweets_flood.ipynb",
        "file": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/recent_tweets_flood.csv",
    },
}
product = {
    "nb": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/tweets_timeline_flood.ipynb"
}


In [4]:
print('Disaster kind: ', disaster_title(disaster_kind))

Disaster kind:  Flood


## Load the sample of recent tweets

In [5]:
df_tweets = pd.read_csv(upstream[disaster_kind]['file'])
df_tweets.sample(5)

,author_id,tweet_text,created_at,hashtags,users
95334,373450900,RT @AppVoices: Help our neighbors recovering from the flooding disaster in Eastern Kentucky and far Southwest Virginia. If you're near Boon…,2022-08-03 01:20:19+00:00,[],['AppVoices']
14787,1555545568579297281,RT @SikandarSherpao: Our deepest condolences with the families of the shaheeds. We laud the role of the armed forces in the rescue and reli…,2022-08-06 04:45:38+00:00,[],['SikandarSherpao']
143782,34799287,"RT @KentuckyMBB: #BBN, we need you on Tuesday.\n\nJoin us for an open practice at Rupp Arena and the Kentucky Flood Relief Telethon with proc…",2022-07-31 22:51:27+00:00,['BBN'],['KentuckyMBB']
110442,1401199198951907330,"RT @OfficialDGISPR: The wreckage of unfortunate hel which was on flood relief ops found in Musa Goth, Windar, Lasbela. All 6 offrs &amp; sldrs…",2022-08-02 11:46:13+00:00,[],['OfficialDGISPR']
97774,1011121749499539457,"RT @OfficialDGISPR: The wreckage of unfortunate hel which was on flood relief ops found in Musa Goth, Windar, Lasbela. All 6 offrs &amp; sldrs…",2022-08-02 22:33:06+00:00,[],['OfficialDGISPR']


## Extract the locations from the tweet sample

In [6]:
%%time
df_tweets, df_locs_most_common = extract_assign_location(df_tweets, number_most_common=10)

df_tweets.sample(5)

0it [00:00, ?it/s]

20it [00:00, 195.79it/s]

56it [00:00, 287.05it/s]

89it [00:00, 303.82it/s]

125it [00:00, 322.98it/s]

...

143955it [06:59, 372.94it/s]

143993it [07:00, 372.15it/s]

144000it [07:00, 342.76it/s]

CPU times: user 7min 28s, sys: 5.41 s, total: 7min 34s
Wall time: 7min 33s


,author_id,tweet_text,created_at,hashtags,users,location
7736,2780071017,@tan123 Yet those floods are the result of storm surge caused by massive drops in Airpressure during a cyclones passage and monsoon flood water time to reach the delta region.,2022-08-06 06:47:00+00:00,[],['tan123'],[Airpressure]
74692,1274430724976906245,RT @DomesticCEOCat: Totally random post or is it? \nbuckets and mopheads for Kentucky… \n\nIt’s not too late to donate cash or supplies. \n\nhtt…,2022-08-03 20:25:00+00:00,[],['DomesticCEOCat'],[]
124559,245060192,"RT @OfficialDGISPR: A Pakistan army aviation helicopter which was on flood relief operations in Lasbela, Balochistan lost contact with ATC…",2022-08-01 21:19:48+00:00,[],['OfficialDGISPR'],"[Pakistan, Lasbela, Balochistan]"
118636,867654732495675393,RT @KazmiWajahat: Disgusted to see so many tweets against our brave #Pakistan Army officers who were busy in helping the public with flood…,2022-08-02 03:14:28+00:00,['Pakistan'],['KazmiWajahat'],[]
126845,1917579673,RT @ShireenMazari1: News of Pak army helicopter missing during flood relief operations is worrisome. Praying for the safety of all those on…,2022-08-01 19:36:40+00:00,[],['ShireenMazari1'],[]


In [7]:
df_locs_most_common

,location,count
0,Balochistan,14825
1,Kentucky,10215
2,Pakistan,9684
3,Lasbela,6493
4,Sirsa,5560
5,Eastern Kentucky,2195
6,Punjab,1132
7,Rupp Arena,1073
8,South Punjab,926
9,Tank,807


## Predict the class labels

In [8]:
# Get the vectorizer
with open(upstream["vectorizer"]["vectorizer"], 'rb') as f:
    vectorizer = pickle.load(f)

In [9]:
# Get the classification model
with open(upstream["category_classification_models"]["model_lr"], 'rb') as f:
    classifier = pickle.load(f)

In [10]:
%%time
# Predict the class labels
class_label = classifier.predict(vectorizer.transform(df_tweets['tweet_text']))
df_tweets['class_label'] = class_label

df_tweets.sample(5)

CPU times: user 34 s, sys: 132 ms, total: 34.1 s
Wall time: 34.1 s


,author_id,tweet_text,created_at,hashtags,users,location,class_label
117583,79811356,RT @ShireenMazari1: News of Pak army helicopter missing during flood relief operations is worrisome. Praying for the safety of all those on…,2022-08-02 04:24:01+00:00,[],['ShireenMazari1'],[],sympathy_and_support
38460,1083237879248084992,A flood watch is up for our region again today as very heavy rainfall is likely. Make sure you stay weather aware as high water issues could show up. #wkyt #kywx #FAWD #flood #watch https://t.co/oPrVYyTbZi,2022-08-05 08:33:09+00:00,"['wkyt', 'kywx', 'FAWD', 'flood', 'watch']",[],[],caution_and_advice
60022,239060658,"i made more goats milk soaps last night. they are not like “i bought it at a farmers market” level good but if anyone wants one, just send me a screenshot of u donating something to eky flood relief and ur address and I’ll mail u a bar this weekend! https://t.co/mzuyIJNtcF",2022-08-04 13:48:37+00:00,[],[],[],not_humanitarian
100146,1341938435293118464,"RT @JamiatPK: We are saddened to hear the news of Military Helicopter, tasked for providing relief to flood-affected areas of Pakistan, was…",2022-08-02 21:41:09+00:00,[],['JamiatPK'],[Pakistan],rescue_volunteering_or_donation_effort
141408,1196059889459769345,RT @NadeemAkhtarSo1: APP LARKANA:\nLARKANA: July 31 -Rescue personnel shifting the flood affected people to the safer places by boat during…,2022-08-01 01:09:17+00:00,[],['NadeemAkhtarSo1'],[],rescue_volunteering_or_donation_effort


## Plot the intensity for each catagory, and for a particular location

### Prepare the time series grouped by disaster class label 

In [11]:
df_tweets['created_at'] = pd.to_datetime(df_tweets['created_at'], errors='coerce')
df_plot = df_tweets.copy()

df_plot['created_at'] = df_plot['created_at'].dt.floor('8H') # Round to 8 hours
df_plot['count'] = 1
df_group_plot = df_plot[['created_at', 'class_label', 'count']].groupby(['created_at', 'class_label']).count().reset_index()
df_group_plot['created_at'] = pd.to_datetime(df_group_plot['created_at'])
df_group_plot.head()

,created_at,class_label,count
0,2022-07-31 16:00:00+00:00,caution_and_advice,37
1,2022-07-31 16:00:00+00:00,infrastructure_and_utility_damage,4
2,2022-07-31 16:00:00+00:00,injured_or_dead_people,135
3,2022-07-31 16:00:00+00:00,not_humanitarian,12
4,2022-07-31 16:00:00+00:00,other_relevant_information,122


### Prepare the time series grouped by location label 

In [12]:
df_explode = df_tweets.copy()

most_common10 = df_locs_most_common.head(10)['location'].tolist()

df_explode = df_explode.explode('location').reset_index(drop=True)
df_explode.dropna(inplace=True)
df_explode['created_at'] = df_explode['created_at'].dt.floor('8H') # Round to 8 hours
df_explode['count'] = 1
df_group_explode = df_explode[['created_at', 'location', 'count']].groupby(
    ['created_at', 'location']
).count().reset_index()
df_group_explode = df_group_explode[df_group_explode['location'].isin(most_common10)]
df_group_explode.head()

,created_at,location,count
3,2022-07-31 16:00:00+00:00,Balochistan,9
12,2022-07-31 16:00:00+00:00,Eastern Kentucky,16
23,2022-07-31 16:00:00+00:00,Kentucky,127
33,2022-07-31 16:00:00+00:00,Pakistan,56
35,2022-07-31 16:00:00+00:00,Punjab,4


### Find the top 10 locations in the tweets sample

In [13]:
top10_locs = list(df_group_explode['location'].unique())
top10_locs

['Balochistan',
 'Eastern Kentucky',
 'Kentucky',
 'Pakistan',
 'Punjab',
 'Rupp Arena',
 'South Punjab',
 'Lasbela',
 'Sirsa',
 'Tank']

In [14]:
chart = alt.Chart(df_group_explode[df_group_explode['location'].isin(top10_locs)]).mark_line(size=2).encode(
    x='created_at:T',
    y='count:Q',
    color='location:N',
    tooltip=['location', 'count']
)

chart.properties(
    width=500,
    height=500
).properties(
    title=disaster_title(disaster_kind) + ' disasters in the world, interactive chart'
).interactive()

alt.Chart(...)

## Combine the most important locations per category

In [15]:
df_top_locations = top_locations(df_tweets, top10_locs)
df_top_locations.sample(5)

,created_at,class_label,locations,count
235,2022-08-04 10:00:00+00:00,other_relevant_information,"[Eastern Kentucky, Kentucky, Pakistan]",26
355,2022-08-06 14:00:00+00:00,other_relevant_information,"[Balochistan, Kentucky]",60
124,2022-08-02 14:00:00+00:00,caution_and_advice,[Kentucky],1
356,2022-08-06 14:00:00+00:00,rescue_volunteering_or_donation_effort,"[Balochistan, Kentucky, Sirsa]",529
39,2022-08-01 10:00:00+00:00,infrastructure_and_utility_damage,[Kentucky],2


### Define the class labels we are interested in

In [16]:
class_labels = [
    'displaced_people_and_evacuations',
    'injured_or_dead_people',
    'rescue_volunteering_or_donation_effort',
    'requests_or_urgent_needs',
    'sympathy_and_support',
    'infrastructure_and_utility_damage',
    'caution_and_advice'
]

In [17]:
chart = alt.Chart(df_top_locations[df_top_locations['class_label'].isin(class_labels)]).mark_line(size=2).encode(
    x='created_at:T',
    y='count:Q',
    color='class_label:N',
    tooltip=['class_label:N', 'locations', 'count']
)

chart.properties(
    width=500,
    height=500
).properties(
    title=disaster_title(disaster_kind) + ' disasters in the world, interactive chart'
)

alt.Chart(...)

## Show sample tweets per most important locations above

In [18]:
df_top_location_tweets = top_locations_tweets(df_tweets, top10_locs)
df_top_location_tweets.sample(5)

,created_at,class_label,tweet_text,locations,count
47,2022-08-01 12:00:00+00:00,not_humanitarian,"[RT @dxena2: @GOP Remember who won't do anything about climate change as states flood, or burn or run out of water - Republicans! Remember…, RT @neelomalik786: Despite the Netherlands being a country at one of the lowest altitudes, and the rain being frequent, the country do...",[Pakistan],1
119,2022-08-02 12:00:00+00:00,injured_or_dead_people,"[RT @nytimes: Kentucky’s governor confirmed on Monday that the death toll from last week’s floods had risen to 37 as rainstorms continued to…, RT @the_hindu: A top #Pakistani army general and five senior military officials were killed when their helicopter crashed while on a ...",[Kentucky],5
225,2022-08-04 06:00:00+00:00,injured_or_dead_people,"[#people #RESCUED TEAM COVERAGE: 3 people still missing after EKY flooding; death toll remains unchanged - WKYT https://t.co/WDQx0i6qDq Gov. Beshear is encouraging people working to repair flood damage in extreme heat to take breaks., RT @TheCivilEyes: Torrential rains and fl...",[Balochistan],153
305,2022-08-05 14:00:00+00:00,infrastructure_and_utility_damage,"[St. Proclus teaches: Come, consider this new and wonderful deluge, greater and more important than the flood of Noah’s day. Then the water of the flood destroyed the human race, but now the water of Baptism has recalled the dead to life by the power of the one who baptized. ...",[Balochistan],6
239,2022-08-04 12:00:00+00:00,not_humanitarian,"[i made more goats milk soaps last night. they are not like “i bought it at a farmers market” level good but if anyone wants one, just send me a screenshot of u donating something to eky flood relief and ur address and I’ll mail u a bar this weekend! https://t.co/mzuyIJNtcF, ...",[Kentucky],6


In [19]:
show_top_locations(df_top_location_tweets, class_labels, disaster_kind)

alt.VConcatChart(...)

## Visualize the locations and relationship between them as a network

### Select the top 20 locations for this disaster

In [20]:
loc20 = get_top_locations_flatten(df_tweets, 20)
loc20

['Balochistan',
 'Kentucky',
 'Pakistan',
 'Lasbela',
 'Sirsa',
 'Eastern Kentucky',
 'Punjab',
 'Rupp Arena',
 'South Punjab',
 'Tank',
 'Sudan',
 'Kentucky’s',
 '@GovernmentKP',
 'Floodwater',
 'Quetta',
 'Sindh',
 'Orangi Town 5',
 'Isom, Ky',
 'Helicopter',
 'UK']

### Get the tweets for just the top 20 locations

In [21]:
df_loc_graph = get_location_graph_data(df_tweets, loc20)
df_loc_graph.sample(10)

,class_label,location_x,location_y,count
1,injured_or_dead_people,Pakistan,Balochistan,51
27,rescue_volunteering_or_donation_effort,Sindh,Pakistan,69
21,rescue_volunteering_or_donation_effort,Punjab,Balochistan,69
38,sympathy_and_support,Lasbela,Helicopter,168
17,rescue_volunteering_or_donation_effort,Kentucky,Eastern Kentucky,22
22,rescue_volunteering_or_donation_effort,Quetta,Balochistan,40
31,rescue_volunteering_or_donation_effort,South Punjab,Sindh,2
37,sympathy_and_support,Lasbela,Balochistan,169
5,other_relevant_information,Helicopter,Balochistan,1
30,rescue_volunteering_or_donation_effort,South Punjab,Punjab,814


In [22]:
plot_disaster_mentions(df_loc_graph, 'caution_and_advice', 'Caution And Advice')

'No records found for class caution_and_advice'

### Get the most frequent location pair

In [23]:
loc_pairs = top_location_pairs(df_loc_graph, 'caution_and_advice', top_pairs_count=1)
loc_pairs

[[]]

### Show a sample for this location pair

In [24]:
show_tweets_locations(df_tweets, class_label='caution_and_advice' , locations=loc_pairs[0], sample=5)

'No location pairs provided'

In [25]:
plot_disaster_mentions(df_loc_graph, 'displaced_people_and_evacuations', 'Displaced People And Evacuations')

'No records found for class displaced_people_and_evacuations'

### Get the most frequent location pair

In [26]:
loc_pairs = top_location_pairs(df_loc_graph, 'displaced_people_and_evacuations', top_pairs_count=1)
loc_pairs

[[]]

### Show a sample for this location pair

In [27]:
show_tweets_locations(df_tweets, class_label='displaced_people_and_evacuations' , locations=loc_pairs[0], sample=5)

'No location pairs provided'

In [28]:
plot_disaster_mentions(df_loc_graph, 'injured_or_dead_people', 'Injured Or Dead People')

alt.LayerChart(...)

### Get the most frequent location pair

In [29]:
loc_pairs = top_location_pairs(df_loc_graph, 'injured_or_dead_people', top_pairs_count=1)
loc_pairs

[['Pakistan', 'Balochistan']]

### Show a sample for this location pair

In [30]:
show_tweets_locations(df_tweets, class_label='injured_or_dead_people' , locations=loc_pairs[0], sample=5)

,tweet_text,location
0,"RT @BNNPK: BREAKING: Death toll hit to 170 as four more people died due to rains and floods in Balochistan, According to Pakistan Disaster…","[Balochistan, Pakistan]"
1,"BREAKING: Death toll hit to 170 as four more people died due to rains and floods in Balochistan, According to Pakistan Disaster Management Authorities. 72 men, 43 women and 55 children were killed in Balochistan due to rains and flood related incidents. https://t.co/NmSSv9OL6R","[Balochistan, Pakistan, Balochistan]"
2,"BREAKING: Death toll hit to 170 as four more people died due to rains and floods in Balochistan, According to Pakistan Disaster Management Authorities. 72 men, 43 women and 55 children were killed in Balochistan due to rains and flood related incidents. https://t.co/NmSSv9OL6R","[Balochistan, Pakistan, Balochistan]"
3,The death toll in flood-affected districts of Balochistan has risen to 136 while 70 persons have been injured.\n#Pakistan #Balochistan #balochistanfloods #floods https://t.co/2BtsCoUZSU,"[Balochistan, Pakistan, Balochistan]"
4,RT @ThePakDaily: The death toll in flood-affected districts of Balochistan has risen to 136 while 70 persons have been injured.\n#Pakistan #…,"[Balochistan, Pakistan]"


In [31]:
plot_disaster_mentions(df_loc_graph, 'rescue_volunteering_or_donation_effort',
                       'Rescue Volunteering Or Donation Effort')

alt.LayerChart(...)

### Get the most frequent location pair

In [32]:
loc_pairs = top_location_pairs(df_loc_graph, 'rescue_volunteering_or_donation_effort', top_pairs_count=1)
loc_pairs

[['Pakistan', 'Balochistan']]

### Show a sample for this location pair

In [33]:
show_tweets_locations(df_tweets, class_label='rescue_volunteering_or_donation_effort',
                      locations=loc_pairs[0], sample=5)

,tweet_text,location
0,"RT @OfficialDGISPR: A Pakistan army aviation helicopter which was on flood relief operations in Lasbela, Balochistan lost contact with ATC…","[Pakistan, Lasbela, Balochistan]"
1,"RT @OfficialDGISPR: A Pakistan army aviation helicopter which was on flood relief operations in Lasbela, Balochistan lost contact with ATC…","[Pakistan, Lasbela, Balochistan]"
2,"RT @OfficialDGISPR: A Pakistan army aviation helicopter which was on flood relief operations in Lasbela, Balochistan lost contact with ATC…","[Pakistan, Lasbela, Balochistan]"
3,"RT @OfficialDGISPR: A Pakistan army aviation helicopter which was on flood relief operations in Lasbela, Balochistan lost contact with ATC…","[Pakistan, Lasbela, Balochistan]"
4,"We are deeply saddened by the tragic crash of a Military Helicopter, tasked for providing relief to flood-affected areas of Pakistan, in Lasbela, Balochistan. https://t.co/6pYiJqP9Od","[Pakistan, Lasbela, Balochistan]"


In [34]:
plot_disaster_mentions(df_loc_graph, 'requests_or_urgent_needs',
                       'requests_or_urgent_needs')

'No records found for class requests_or_urgent_needs'

### Get the most frequent location pair

In [35]:
loc_pairs = top_location_pairs(df_loc_graph, 'requests_or_urgent_needs', top_pairs_count=1)
loc_pairs

[[]]

### Show a sample for this location pair

In [36]:
show_tweets_locations(df_tweets, class_label='requests_or_urgent_needs',
                      locations=loc_pairs[0], sample=5)

'No location pairs provided'

In [37]:
plot_disaster_mentions(df_loc_graph, 'sympathy_and_support',
                       'sympathy_and_support')

alt.LayerChart(...)

### Get the most frequent location pair

In [38]:
loc_pairs = top_location_pairs(df_loc_graph, 'sympathy_and_support', top_pairs_count=1)
loc_pairs

[['Helicopter', 'Balochistan']]

### Show a sample for this location pair

In [39]:
show_tweets_locations(df_tweets, class_label='sympathy_and_support',
                      locations=loc_pairs[0], sample=5)

,tweet_text,location
0,A great loss of nation !!\n\nMay Allah elevate the status of our martyrs lost in a Helicopter crash during a flood relief operation in Balochistan near Lasbela.\n\n #WeSaluteOurHeroes https://t.co/sQEFwYfZEw,"[Helicopter, Balochistan, Lasbela]"
1,A great loss of nation !!\n\nMay Allah elevate the status of our martyrs lost in a Helicopter crash during a flood relief operation in Balochistan near Lasbela.\n#WeSaluteOurHeroes https://t.co/0gV59jLXoP,"[Helicopter, Balochistan, Lasbela]"
2,A great loss of nation !!\n\nMay Allah elevate the status of our martyrs lost in a Helicopter crash during a flood relief operation in Balochistan near Lasbela.\n\n #WeSaluteOurHeroes,"[Helicopter, Balochistan, Lasbela]"
3,A great loss of nation !!\nMay Allah elevate the status of our martyrs lost in a Helicopter crash during a flood relief operation in Balochistan near Lasbela. \n #WeSaluteOurHeroes https://t.co/qBTmQ2m6m2,"[Helicopter, Balochistan, Lasbela]"
4,A great loss of nation !!\n\nMay Allah elevate the status of our martyrs lost in a Helicopter crash during a flood relief operation in Balochistan near Lasbela. \n.. \n\n #WeSaluteOurHeroes,"[Helicopter, Balochistan, Lasbela]"


In [40]:
plot_disaster_mentions(df_loc_graph, 'infrastructure_and_utility_damage',
                       'infrastructure_and_utility_damage')

alt.LayerChart(...)

### Get the most frequent location pair

In [41]:
loc_pairs = top_location_pairs(df_loc_graph, 'infrastructure_and_utility_damage', top_pairs_count=1)
loc_pairs

[['Punjab', 'Balochistan']]

### Show a sample for this location pair

In [42]:
show_tweets_locations(df_tweets, class_label='infrastructure_and_utility_damage',
                      locations=loc_pairs[0], sample=5)

,tweet_text,location
0,RT @SaadUrR11747632: Flood has destroyed Balochistan and south Punjab very badly\nOver 300 people had lost their lives\n@AlkhidmatOrg and its…,"[Balochistan, Punjab]"
